<a href="https://colab.research.google.com/github/mariliaribeiro/hypothesis_tests/blob/main/testes_hipoteses_exerc%C3%ADcios_refatorado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
#@title Instalação de bibliotecas
%%R 
install.packages("distributions3")

if (!require("devtools")) install.packages("devtools")
devtools::install_github("mkuhn/dict")

In [91]:
#@title Declara uso das bibliotecas
%%R 
library(distributions3)
library(dict)

In [92]:
%%R
options(scipen=999)

In [93]:
#@title Funções base: Z crítico, T crítico e p-value
%%R 

get_z_critical <- function(z_stat, h1_signal='<>', alpha=0.05){
  if (h1_signal == '>') {
      z_critical <- qnorm(p=alpha, lower.tail=FALSE)
      z_accept_h1 <- z_stat > z_critical
  } else if (h1_signal == '<') {
    z_critical <- qnorm(p=alpha, lower.tail=TRUE)
    z_accept_h1 <- z_stat < z_critical
  } else {
      z_critical <- qnorm(p=alpha/2, lower.tail=FALSE)
      z_accept_h1 <- (abs(z_stat) > abs(z_critical)) | (-abs(z_stat) < -abs(z_critical))
  }
  cat('\nz_stat = ', z_stat, '  z_critical = ', z_critical, 'Accept H1? R.: ', z_accept_h1)
  
  return(z_critical)
}


# https://www.statology.org/t-critical-value-r/
get_t_critical <- function(t_stat, df, h1_signal='<>', alpha=0.05){
  if (h1_signal == '>') {
      t_critical <- qt(p=alpha, df=df, lower.tail=FALSE)
      t_accept_h1 <- t_stat > t_critical
  } else if (h1_signal == '<') {
    t_critical <- qt(p=alpha, df=df, lower.tail=TRUE)
    t_accept_h1 <- t_stat < t_critical
  } else {
      t_critical <- qt(p=alpha/2, df=df, lower.tail=FALSE)
      t_accept_h1 <- (abs(t_stat) > abs(t_critical)) | (-abs(t_stat) < -abs(t_critical))
  }
  cat('\nt_stat = ', t_stat, '  t_critical = ', t_critical, 'Accept H1? R.: ', t_accept_h1)
  
  return(t_critical)
}


# necessário instanciar 
# Z <- Normal(0, 1)
# T <- StudentsT(df=df) 
# method = Z ou T

get_p_value <- function(stat_value, method,h1_signal='<>', alpha=0.05){
  if (h1_signal == '>') {
    p_value <- 1 - cdf(method, stat_value)
  } else if (h1_signal == '<') {
      p_value <- cdf(method, stat_value)
  } else {
      p_value <- 2 * cdf(method, -abs(stat_value))
  }  
  p_accept_h1 <- p_value <= alpha
  cat('\np_value <= alpha? p-value =', p_value, 'alpha =', alpha, '  Accept H1? R.: ', p_accept_h1)  
  
  return(p_value)
}

# Testes para uma amostra

In [94]:
#@title Teste Z para média de uma amostra
# https://cran.r-project.org/web/packages/distributions3/vignettes/one-sample-z-test.html

#@markdown `one_sample_z_test(n, sd, mean_sample, mean_0, h1_signal='<>', alpha=0.05)`



%%R
# install.packages("distributions3")
# library(distributions3)

one_sample_z_test <- function(n, sd, mean_sample, mean_0, h1_signal='<>', alpha=0.05){
  # h1_signal <- '<', '>','<>'
  z_stat <- (mean_sample - mean_0) / (sd / sqrt(n))
  z_critical <- get_z_critical(z_stat, h1_signal,alpha)  
  Z <- Normal(0, 1)
  p_value <- get_p_value(z_stat, Z, h1_signal,alpha)

  E = (abs(z_critical) * sd / sqrt(n))
  ic_min <- mean_sample - E
  ic_max <- mean_sample + E
  ic_accept_h1 = !(mean_0 > ic_min & mean_0 < ic_max)
  cat('\nE = ', E, 'IC = [', ic_min, '; ', ic_max, ']', 'Accept H1? R.: ', ic_accept_h1)
}

x <- c(3, 7, 11, 0, 7, 0, 4, 5, 6, 2)
dp <- 2
mean_0 <- 3

# H0: mean = 3
# H1: mean <> 3

one_sample_z_test(n=length(x), sd=dp, mean_sample=mean(x), mean_0=mean_0, h1_signal='<>', alpha=0.05)


z_stat =  2.371708   z_critical =  1.959964 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.01770607 alpha = 0.05   Accept H1? R.:  TRUE
E =  1.23959 IC = [ 3.26041 ;  5.73959 ] Accept H1? R.:  TRUE

In [95]:
#@title Teste Z para proporção de uma amostra

#@markdown `one_sample_z_proportion_test(n, pi, pi_0, h1_signal='<>', alpha=0.05)`

%%R
# install.packages("distributions3")
# library(distributions3)


one_sample_z_proportion_test <- function(n, pi, pi_0, h1_signal='<>', alpha=0.05){
  # h1_signal <- '<', '>','<>'
  cat('n * pi > 5 = ', (n * pi > 5))
  cat('\n')
  cat('n * (1 - pi) > 5 = ', (n * (1 - pi) > 5))
  cat('\n')
  
  z_stat <- (pi - pi_0) / sqrt(pi_0 * (1 - pi_0) / n)
  z_critical <- get_z_critical(z_stat, h1_signal,alpha)  
  Z <- Normal(0, 1)
  p_value <- get_p_value(z_stat, Z, h1_signal,alpha)

  E = abs(z_critical) * sqrt(pi * (1 - pi) / n)
  ic_min <- pi - E
  ic_max <- pi + E 
  ic_accept_h1 = !(pi_0 > ic_min & pi_0 < ic_max )
  cat('\nE = ', E, 'IC = [', ic_min, '; ', ic_max, ']', 'Accept H1? R.: ', ic_accept_h1)
}



n <- 62
pi <- 17 / 62
pi_0 <- 0.2
# H0: p = 0.2
# HA: p ≠ 0.2

one_sample_z_proportion_test(n=n, pi=pi, pi_0=pi_0)

n * pi > 5 =  TRUE
n * (1 - pi) > 5 =  TRUE

z_stat =  1.460501   z_critical =  1.959964 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.1441523 alpha = 0.05   Accept H1? R.:  FALSE
E =  0.111043 IC = [ 0.1631505 ;  0.3852366 ] Accept H1? R.:  FALSE

In [96]:
#@title Teste t para média de uma amostra
# https://cran.r-project.org/web/packages/distributions3/vignettes/one-sample-t-test.html

#@markdown `one_sample_t_test(n, sd, mean_sample, mean_0, h1_signal='<>', alpha=0.05)`

%%R
# install.packages("distributions3")
# library(distributions3)

one_sample_t_test <- function(n, sd, mean_sample, mean_0, h1_signal='<>', alpha=0.05){
  # h1_signal <- '<', '>','<>'
  t_stat <- (mean_sample - mean_0) / (sd / sqrt(n))  
  df = n -1
  cat('\ndf = ', df)
  t_critical <- get_t_critical(t_stat, df, h1_signal, alpha)
  T <- StudentsT(df=df) 
  p_value <- get_p_value(t_stat, T, h1_signal,alpha)

  E = abs(t_critical) *  sd / sqrt(n)
  ic_min <- mean_0 - E
  ic_max <- mean_0 + E
  ic_accept_h1 = !(mean_0 > ic_min & mean_0 < ic_max)
  cat('\nE = ', E, 'IC = [', ic_min, '; ', ic_max, ']', 'Accept H1? R.: ', ic_accept_h1)
}

x <- c(3, 7, 11, 0, 7, 0, 4, 5, 6, 2)
mean_0 <- 3
# H0:μ=3
# HA:μ≠3

one_sample_t_test(n=length(x), sd=sd(x), mean_sample=mean(x), mean_0=mean_0)


df =  9
t_stat =  1.378916   t_critical =  2.262157 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.2012211 alpha = 0.05   Accept H1? R.:  FALSE
E =  2.4608 IC = [ 0.5392 ;  5.4608 ] Accept H1? R.:  FALSE

#Duas amostras

In [97]:
#@title Teste Z para média de duas amostras
# https://cran.r-project.org/web/packages/distributions3/vignettes/two-sample-z-test.html

#@markdown `two_sample_z_test(n_1, n_2, mean_1, mean_2, var_1, var_2, delta_0=0, h1_signal='<>', alpha=0.05)`

%%R
# install.packages("distributions3")
# library(distributions3)

two_sample_z_test <- function(n_1, n_2, mean_1, mean_2, var_1, var_2, delta_0=0, h1_signal='<>', alpha=0.05){
  # h1_signal <- '<', '>','<>'
  
  z_stat <- (mean_1 - mean_2 - delta_0)/sqrt((var_1/n_1) + (var_2/n_2))
  z_critical <- get_z_critical(z_stat, h1_signal,alpha)  
  Z <- Normal(0, 1)
  p_value <- get_p_value(z_stat, Z, h1_signal,alpha)

  E = abs(z_critical) * sqrt((var_1/n_1) + (var_2/n_2))
  ic_min <- (mean_1 - mean_2) - E
  ic_max <- (mean_1 - mean_2) + E
  ic_accept_h1 = !(0 > ic_min & 0 < ic_max)
  cat('\nE = ', E, 'IC = [', ic_min, '; ', ic_max, ']', 'Accept H1? R.: ', ic_accept_h1)
  
  # return(p_value)
}

biology <- c(3, 7, 11, 0, 7, 0, 4, 5, 6, 2, 4, 7, 2, 9) # 1
english <- c(8, 5, 4, 10, 4, 5, 7, 2, 6, 1, 2, 7, 0, 6, 4, 12, 5, 2) # 2

n_biology <- 14
n_english <- 18

var_biology <- 3
var_english <- 2

# H0: mean_1 - mean_2 = 0 
# H1: mean_1 - mean_2 <> 0 


two_sample_z_test(n_1=n_biology, n_2=n_english, mean_1=mean(biology), mean_2=mean(english), var_1=var_biology, var_2=var_english, delta_0=0, h1_signal='<>', alpha=0.05)


z_stat =  -0.3756527   z_critical =  1.959964 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.7071751 alpha = 0.05   Accept H1? R.:  FALSE
E =  1.118033 IC = [ -1.332319 ;  0.9037476 ] Accept H1? R.:  FALSE

In [98]:
#@title Teste Z para proporção de duas amostras

#@markdown `two_sample_z_proportion_test(n_1, n_2, p_1, p_2, h1_signal='<>', alpha=0.05)`

%%R
# install.packages("distributions3")
# library(distributions3)

two_sample_z_proportion_test <- function(n_1, n_2, p_1, p_2, h1_signal='<>', alpha=0.05){
  # h1_signal <- '<', '>','<>'
  p <- ((n_1 * p_1) + (n_2 * p_2))/ (n_1 + n_2)
  cat('p_^ = ', p,'\n')
  z_stat <- (p_1 - p_2) / sqrt(((p * (1 - p))/ n_1) + ((p * (1 - p))/ n_2))  
  z_critical <- get_z_critical(z_stat, h1_signal,alpha)  
  Z <- Normal(0, 1)
  p_value <- get_p_value(z_stat, Z, h1_signal,alpha)

  E = abs(z_critical) * sqrt(((p * (1 - p))/ n_1) + ((p * (1 - p))/ n_2))
  ic_min <- (p_1 - p_2) - E
  ic_max <- (p_1 - p_2) + E  
  ic_accept_h1 = !(0 > ic_min & 0 < ic_max)
  cat('\nE = ', E, 'IC = [', ic_min, '; ', ic_max, ']', 'Aceept H1? R.: ', ic_accept_h1)
}

two_sample_z_proportion_test(n_1=134, n_2=145, p_1=122/134, p_2=127/145, h1_signal='>', alpha=0.1)

p_^ =  0.8924731 

z_stat =  0.9316969   z_critical =  1.281552 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.1757466 alpha = 0.1   Accept H1? R.:  FALSE
E =  0.04757271 IC = [ -0.01298702 ;  0.0821584 ] Aceept H1? R.:  FALSE

In [99]:
#@title Teste t para média de duas amostras pareadas (dependentes)

#@markdown `t.test(antes, depois, paired=TRUE, conf.level=0.99, alternative='two.sided')`

#@markdown paired = FALSE -> por default o teste admite que os grupos não sejam pareados

#@markdown conf.level = 0.95 -> por default o teste admite um nível de confiança de 95% (ou α= 5%)

#@markdown alternative = two.sided - > por default o teste é bicaudal, podemos escolher também “greater” ou “less”



%%R
antes  <- c(16, 15, 11, 20, 19, 14, 13, 15, 14, 16)
depois <- c(13, 13, 12, 16, 16, 11, 10, 15, 9, 16)
 

t.test(antes, depois, paired=TRUE, conf.level=0.99, alternative='two.sided')


	Paired t-test

data:  antes and depois
t = 3.6006, df = 9, p-value = 0.005743
alternative hypothesis: true difference in means is not equal to 0
99 percent confidence interval:
 0.2143177 4.1856823
sample estimates:
mean of the differences 
                    2.2 



In [100]:
#@title Teste t para média de duas amostras não pareadas (independentes)

#@markdown `two_sample_t_test_heterocedastico(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=0.05)`

#@markdown `two_sample_t_test_homocedastico(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=0.05)`

#@markdown Como saber se as duas populações têm variâncias iguais (homocedásticas) ou não(heterocedásticas)? Utilizar teste F.

%%R
# install.packages("distributions3")
# library(distributions3)

two_sample_t_test_heterocedastico <- function(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=0.05){
  # h1_signal <- '<', '>','<>'
  t_stat <- (mean_1 - mean_2) / sqrt((var_1/n_1) + (var_2/n_2))
  
  if ((n_1 - 1) <= (n_2 - 1)) {
      df <- (n_1 - 1)
  } else {
      df <- (n_2 - 1)
  }
  t_critical <- get_t_critical(t_stat, df, h1_signal, alpha)
  T <- StudentsT(df=df) 
  p_value <- get_p_value(t_stat, T, h1_signal,alpha)
  
  E <- abs(t_critical) *  sqrt((var_1/n_1) + (var_2/n_2))
  ic_min <- (mean_1 - mean_2) - E
  ic_max <- (mean_1 - mean_2) + E  
  ic_accept_h1 = !(0 > ic_min & 0 < ic_max)
  cat('\nE = ', E, 'IC = [', ic_min, '; ', ic_max, ']', 'Accept H1? R.: ', ic_accept_h1)
  
  # return(p_value)
}

two_sample_t_test_homocedastico <- function(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=0.05){
  # h1_signal <- '<', '>','<>'
  s_comb <- (((n_1 - 1) * var_1) + ((n_2 - 1) * var_2)) / ((n_1 - 1) + (n_2 - 1))
  t_stat <- (mean_1 - mean_2) / sqrt((s_comb/n_1) + (s_comb/n_2))
  
  df = n_1 + n_2 - 2
  cat('\ndf = ', df, 's_comb = ', s_comb)
  t_critical <- get_t_critical(t_stat, df, h1_signal, alpha)

  T <- StudentsT(df=df) 
  p_value <- get_p_value(t_stat, T, h1_signal,alpha)

  E <- abs(t_critical) *  sqrt((s_comb/n_1) + (s_comb/n_2))
  ic_min <- (mean_1 - mean_2) - E
  ic_max <- (mean_1 - mean_2) + E
  ic_accept_h1 = !(0 > ic_min & 0 < ic_max)
  cat('\nE = ', E, 'IC = [', ic_min, '; ', ic_max, ']', 'Accept H1? R.: ', ic_accept_h1)
  
  # return(p_value)
}

In [101]:
#@title Teste F para variâncias de duas amostras
# https://bookdown.org/matheusogonzaga/apostila_r2/inferencia.html#teste-f-para-compara%C3%A7%C3%A3o-de-duas-vari%C3%A2ncias

#@markdown `f_test_to_var(n_1, n_2, var_1, var_2, alpha=0.05)`

#@markdown homocedática -> igualdade entre duas variâncias

#@markdown heterocedática -> diferênça entre duas variâncias


%%R

f_test_to_var <- function(n_1, n_2, var_1, var_2, alpha=0.05) {
  F <- var_1/ var_2

  gl1 <- n_1 - 1
  gl2 <- n_2 - 1  
  Fl <- (1/qf((alpha/2),gl2,gl1,lower.tail=F))
  Fr <- qf((alpha/2),gl1,gl2,lower.tail=F)

  if (F < Fl | F > Fr){ 
      result <- 'Aceita H1'
  } else { 
      result <- 'Aceita H0'
  }

  cat('F = ', F, '\nFl = ', Fl, '\nFr = ', Fr, '\nResultado = ', result)
}

n_1 <- 15
n_2 <- 10

var_1 <- 0.03
var_2 <- 0.19

f_test_to_var(n_1, n_2, var_1, var_2, alpha=0.05)

F =  0.1578947 
Fl =  0.3115944 
Fr =  3.797952 
Resultado =  Aceita H1

# Exemplos e Exercícios

## Exercícios Teste Z para média de uma amostra

In [102]:
#@markdown 1. (Adaptado de Stevenson, 2001) Suponha que queiramos avaliar a afirmação de um fabricante, de que seus pneus radiais suportam uma quilometragem de 40000 milhas, no mínimo. Para uma amostra aleatória de 49 pneus observou-se uma média de duração de 38000 milhas. Sabe-se que o desvio padrão populacional da duração dos pneus é de 3500 milhas. Utilize um nível de significância de 0,03. O que você pode concluir? Obtenha a conclusão utilizando, também, o valor p. 

#@markdown > H0: μ>=4000 vs HA: μ<40000 \
#@markdown > zteste= - 4 \
#@markdown > z0,03= -1,88 \
#@markdown > Como zteste< z0,03, deve-se rejeitar H0. \
#@markdown > Conclusão: A duração média dos pneus radiais é inferior a 40000 milhas, contradizendo a afirmação do fabricante a um nível de 3% de significância.

#@markdown > Valor p = 0,5-0,4998 = 0,0002 \
#@markdown > Como valor p < 0,03, deve-se rejeitar H0. \
#@markdown > Conclusão: A duração média dos pneus radiais é inferior a 40000 milhas, contradizendo a afirmação do fabricante a um nível de 3% de significância.

%%R

mean_0 <- 40000
n <- 49
mean_sample <- 38000
sd <- 3500
alpha <- 0.03

# H0: >= 40000
# H1: < 40000

one_sample_z_test(n, sd, mean_sample, mean_0, h1_signal='<', alpha=alpha)


z_stat =  -4   z_critical =  -1.880794 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.00003167124 alpha = 0.03   Accept H1? R.:  TRUE
E =  940.3968 IC = [ 37059.6 ;  38940.4 ] Accept H1? R.:  TRUE

In [103]:
#@markdown 2. Um representante de um grupo comunitário informa a um potencial construtor de um shopping, que a renda familiar média mensal nesta área é superior a R$6500. Suponha que, para o tipo de área envolvida, a renda familiar possa ser assumida como seguindo a distribuição normal, e que o desvio padrão pode ser aceito como sendo igual a  = 2000, baseado em um estudo anterior. Para uma amostra aleatória de 55 residências, a renda familiar média encontrada é de R$6300. Verifique se há evidências que confirmem a hipótese do representante do grupo comunitário, considere um nível de 2% de significância. Utilize a região crítica e o valor p para concluir o teste. 

#@markdown > H0: μ<=6500 vs HA: μ>6500 \
#@markdown > zteste= -0,7416 \
#@markdown > Valor p = 0,5+0,2704 = 0,7704 \
#@markdown > Como valor p > 0,02, NÃO deve-se rejeitar H0. \
#@markdown > Conclusão: Conclui-se com 2% de significância que a renda familiar média mensal nesta área é de, no máximo, R$6500. Dessa forma, não há evidências que confirmem a hipótese do representante do grupo comunitário.

%%R

# H0: <= 6500
# H1: > 6500

mean_0 <- 6500
sd <- 2000
n <- 55
mean_sample <- 6300
alpha <- 0.02

one_sample_z_test(n, sd, mean_sample, mean_0, h1_signal='>', alpha=alpha)


z_stat =  -0.7416198   z_critical =  2.053749 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.7708412 alpha = 0.02   Accept H1? R.:  FALSE
E =  553.8549 IC = [ 5746.145 ;  6853.855 ] Accept H1? R.:  FALSE

In [104]:
#@markdown 1. Um estudante de direito quer conferir a alegação de sua professora de que fraudadores condenados passam, em média 12,3 meses na cadeia. Uma amostra aleatória de 35 casos do arquivo judiciário indicou uma média de 11,5 meses. Suponha que o desvio padrão seja sd = 3,8 meses. Adote um nível de 4% de significância. O que o estudante pode concluir a partir dos dados da amostra? Utilize o valor p e o intervalo de confiança. 

#@markdown > H0: μ=12,3 vs HA: μ≠12,3 \
#@markdown > zteste= -1,2455 \
#@markdown > Pelo método do Valor p: \
#@markdown > Valor p = 2x(0,5-0,3944)=0,2112 \
#@markdown > Como valor p > 0,04, NÃO deve-se rejeitar H0. \
#@markdown > Pelo método do intervalo de confiança (já que estamos em um teste bilateral): \
#@markdown > zc=2,05 \
#@markdown > E= 1,3167 meses \
#@markdown > I.C.= [10,1833 meses ; 12,8167 meses] \
#@markdown > Como o valor de mean_0 = 12,3 está compreendido no intervalo de confiança, não deve-se rejeitar H0. Confirmando a conclusão obtida pelo método do valor p. Conclusão: Conclui-se com 4% de significância que o tempo médio que fraudadores condenados passam na cadeia é de 12,3 meses. Dessa forma, há evidências que confirmam a alegação da professora.

%%R

# H0: = 12.3
# H1: <> 12.3

mean_0 <- 12.3
n <- 35
mean_sample <- 11.5
sd <- 3.8
alpha <- 0.04


one_sample_z_test(n, sd, mean_sample, mean_0, h1_signal='<>', alpha=alpha)


z_stat =  -1.24549   z_critical =  2.053749 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.2129515 alpha = 0.04   Accept H1? R.:  FALSE
E =  1.319158 IC = [ 10.18084 ;  12.81916 ] Accept H1? R.:  FALSE

## Teste Z para proporção de uma amostra

In [105]:
#@markdown 1. Um sistema de reservas da companhia Y acusa uma taxa de 7% de não-comparecimento. Adotou-se então um novo processo, pelo qual as reservas são confirmadas no dia anterior ao do vôo, fazendo-se um estudo de 5218 reservas pelo novo sistema, selecionadas aleatoriamente. Se se registraram 333 não-comparecimentos, teste a afirmação de que a taxa de não-comparecimento é menor no novo sistema. O novo sistema se afigura eficiente na redução do não-comparecimento? Utilize um nível de 2% de significância.

#@markdown > Hipótese unilateral à esquerda \
#@markdown > pˆ =0,0638 \
#@markdown > zteste= -1,7553 \
#@markdown > valor p =0,5-0,4608=0,0392 > alpha -> NÃO rejeito a hipótese nula

%%R
cat('p_^ = ', 333/5218, '\n')
one_sample_z_proportion_test(n=5218, pi=333/5218, pi_0=0.07, h1_signal='<', alpha=0.02)

p_^ =  0.06381755 
n * pi > 5 =  TRUE
n * (1 - pi) > 5 =  TRUE

z_stat =  -1.750338   z_critical =  -2.053749 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.04003004 alpha = 0.02   Accept H1? R.:  FALSE
E =  0.006949373 IC = [ 0.05686818 ;  0.07076693 ] Accept H1? R.:  FALSE

In [106]:
#@markdown 2. Em um estudo de 71 fumantes que estavam procurando deixar de fumar utilizando uma terapia especial, 32 não estavam fumando um ano após o tratamento. Ao nível de 0,10 de significância, teste a afirmação de que, dos fumantes que procuram deixar de fumar com aquela terapia, no máximo 25% voltam a fumar um ano após o tratamento. Esses resultados sugerem que a terapia não é eficaz? 

#@markdown > Hipótese unilateral à direita \
#@markdown > pˆ=0,5493 \
#@markdown > zteste= 5,8242 \
#@markdown > valor p =0,5-0,4998=0,0002 < alpha -> Rejeito a hipótese nula

%%R
cat('p_^ = ', (71-32)/71, '\n')
one_sample_z_proportion_test(n=71, pi=(71-32)/71, pi_0=0.25, h1_signal='>', alpha=0.1)

p_^ =  0.5492958 
n * pi > 5 =  TRUE
n * (1 - pi) > 5 =  TRUE

z_stat =  5.824104   z_critical =  1.281552 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.00000000287099 alpha = 0.1   Accept H1? R.:  TRUE
E =  0.0756756 IC = [ 0.4736202 ;  0.6249714 ] Accept H1? R.:  TRUE

## Teste Z para proporção de duas amostras

Exemplo
Uma empresa que presta serviços de assessoria econômica a outras empresas está interessada em
comparar a taxa de reclamações sobre os seus serviços em dois dos seus escritórios em duas cidades
diferentes. Suponha que a empresa tenha selecionado aleatoriamente 134 serviços realizados pelo
escritório da cidade A e foi constatado que em 12 deles houve algum tipo de reclamação. Já do escritório
da cidade B foram selecionados 145 serviços e 18 receberam algum tipo de reclamação. A empresa
deseja saber se estes resultados são suficientes para se concluir que o escritório A apresenta uma taxa de
aprovação maior que o escritório B. Use α = 0,1.
Podemos observar que o parâmetro a ser comparado é a taxa de aprovação dos serviços de dois
escritórios localizados em duas cidades diferentes, ou seja, duas proporções. Vamos considerar a cidade
A como a 1a amostra e a cidade B como a 2a amostra. Os dados da pesquisa disponíveis no enunciado
são:
- n1 = 134 (tamanho da 1a amostra)
- n2 = 145 (tamanho da 2a amostra)
- x1 = 134 - 12 = 122 (número de sucessos da 1a amostra)
- x2 = 145 – 18 = 127 (número de sucessos da 2a amostra)
- p̂1= 122/134 = 0,9104 (proporção de sucessos da 1a amostra)
- p̂2= 127/145 = 0,8759 (proporção de sucessos da 2a amostra)

> A questão a ser respondida é: ”Os resultados obtidos permitem concluir que o escritório A apresenta uma
taxa de aprovação maior que o escritório B?”. A partir daí as hipóteses estatísticas podem ser elaboradas:
H0: p1 < p2 versus HA: p1 > p2 → Hipótese unilateral à direita

- Calcular p chapéu
- Calcular estatística de teste Z
- Obter p-value




In [107]:
%%R
two_sample_z_proportion_test(n_1=134, n_2=145, p_1=122/134, p_2=127/145, h1_signal='>', alpha=0.1)

p_^ =  0.8924731 

z_stat =  0.9316969   z_critical =  1.281552 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.1757466 alpha = 0.1   Accept H1? R.:  FALSE
E =  0.04757271 IC = [ -0.01298702 ;  0.0821584 ] Aceept H1? R.:  FALSE

1. Uma firma especializada em declarações de imposto de renda está interessada em comparar a
qualidade do trabalho em dois de seus escritórios regionais (Sul e Sudeste). Ao selecionar
aleatoriamente amostras de declarações do imposto de renda preenchidas em cada escritório e
verificar a precisão amostral das declarações, a firma será capaz de estimar a proporção das
declarações preenchidas erroneamente em cada escritório. A firma está interessada em verificar se
a proporção de declarações preenchidas erroneamente é diferente nos dois escritórios. Use α = 0,03.

| Escritório regional | Tamanho amostral | Declarações preenchidas erroneamente |
| ---------| ----------| ------------|
| Sul | 240 | 21 |
| Sudeste | 350 | 59 |

> - Hipótese bilateral
> - p̅= 0,1356
- z = 2,8258
> - Valor p = 2x(0,5-0,4977)=0,0046
> - Como valor p < alpha, podemos rejeitar a hipótese nula, ou seja, a proporção de declarações preenchidas erroneamente difere entre os dois escritórios.

> - Intervalo de confiança: \
> z=2,17 \
> [-0,1434 ; -0,0188]

> - Como o intervalo de confiança não contém o zero, podemos rejeitar a hipótese nula. Além disso, pode-se
dizer que a proporção de declarações preenchidas erroneamente pelo escritório da regional sudeste é
significativamente maior do que no escritório da regional sul, com um nível de 97% de confiança.

In [108]:
%%R
two_sample_z_proportion_test(n_1=240, n_2=350, p_1=21/240, p_2=59/350, h1_signal='<>', alpha=0.03)

p_^ =  0.1355932 

z_stat =  -2.825553   z_critical =  2.17009 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.004719914 alpha = 0.03   Accept H1? R.:  TRUE
E =  0.06226475 IC = [ -0.1433362 ;  -0.01880668 ] Aceept H1? R.:  TRUE

2. Em um estudo 250 homens adultos e 200 mulheres adultas, ambos usuários de internet, foram selecionados aleatoriamente. Entre os homens 10% disseram realizar compras online pelo menos uma vez no mês. Entre as mulheres, esse  percentual foi de 30%. Teste a afirmação de que a proporção de homens usuários  de internet que compram pelo menos uma vez no mês é inferior a de mulheres. Utilize um nível de significância de 7%

> Hipótese unilateral esquerda
> Como valor p < alpha, podemos rejeitar a hipótese nula. Conclui-se com 7% de significância que a proporção de homens usuários de internet que compram pelo  menos uma vez no mês é inferior a de mulheres


In [109]:
%%R
two_sample_z_proportion_test(n_1=250, n_2=200, p_1=0.10, p_2=0.30, h1_signal='<', alpha=0.07)

p_^ =  0.1888889 

z_stat =  -5.385988   z_critical =  -1.475791 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.00000003602394 alpha = 0.07   Accept H1? R.:  TRUE
E =  0.05480113 IC = [ -0.2548011 ;  -0.1451989 ] Aceept H1? R.:  TRUE

## Exercícios Teste t para média de uma amostra

In [110]:
#@markdown 1. (Adaptado de Morettin & Bussab) Um escritório de investimentos acredita que o rendimento médio das diversas ações que movimenta é de 20%. Uma nova estratégia foi definida com o objetivo de aumentar o rendimento médio das diversas ações que movimenta. Para verificar esta hipótese, tomaram-se 18 empresas ao acaso obtendo-se um rendimento médio de 24,56% e um desvio padrão de 5%. Suponha que a população tenha distribuição aproximadamente normal. Faça o teste estatístico para comprovar que a nova estratégia é realmente eficiente para aumentar o rendimento médio das diversas ações. Use um nível de significância de 1%.

#@markdown > Hipótese unilateral à direita, tteste=3,8692, valor p=0,0005

%%R
one_sample_t_test(n=18, sd=0.05, mean_sample=0.2456, mean_0=0.20, h1_signal='>', alpha=0.01)


df =  17
t_stat =  3.869288   t_critical =  2.566934 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.0006155272 alpha = 0.01   Accept H1? R.:  TRUE
E =  0.03025161 IC = [ 0.1697484 ;  0.2302516 ] Accept H1? R.:  FALSE

In [111]:
#@markdown 2. Uma amostra aleatória de oito pedidos dos arquivos de uma companhia mostra que os pedidos de certa peça de uma máquina forma despachados em: 12 10 17 14 13 18 11 9 dias. Ao nível de 5% de significância , podemos concluir que, em média, tais pedidos são despachados em menos de 10 dias?

#@markdown > Hipótese unilateral à esquerda, média da amostra=13, variância da amostra=10,2857, desvio padrão da amostra=3,2071, tteste=2,6458, valor p=1-0,0125=0,9875.

%%R
x <- c(12, 10, 17, 14, 13, 18, 11, 9)
mean_sample <- mean(x)
sd <- sd(x)
cat('mean_sample = ', mean_sample, 'sd = ', sd, '\n')
one_sample_t_test(n=8, sd, mean_sample, mean_0=10, h1_signal='<', alpha=0.05)

mean_sample =  13 sd =  3.207135 

df =  7
t_stat =  2.645751   t_critical =  -1.894579 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.9834272 alpha = 0.05   Accept H1? R.:  FALSE
E =  2.14825 IC = [ 7.85175 ;  12.14825 ] Accept H1? R.:  FALSE

## Teste t pareado para duas amostras

conf.level = 0.95 -> por default o teste admite um nível de confiança de 95% (ou α= 5%)

alternative = two.sided - > por default o teste é bicaudal, podemos escolher também “greater” ou “less”

Exemplo 1
O Captopril é um remédio destinado a baixar a pressão sistólica. Feito o teste com este
remédio em pacientes, mediram-se suas pressões sistólicas (em mmHg) antes e depois de
tomarem o remédio, com os resultados constantes da tabela a seguir. Pode-se dizer que haja
diferença nas medidas da pressão sistólica antes e depois da ingestão do remédio? Utilize alpha = 5%

Pressão sistólica (em mmHg)

|Antes | Depois|
|--- | --- |
|200 | 191 |
|174 | 170 |
|198 | 177 |
|170 | 167 |
|179 | 159 |

In [112]:
%%R

antes  <- c(200, 174, 198, 170, 179)
depois <- c(191, 170, 177, 167, 159)
t.test(antes, depois, paired=TRUE, conf.level=0.95, alternative='two.sided')


	Paired t-test

data:  antes and depois
t = 2.9573, df = 4, p-value = 0.04167
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
  0.6971732 22.1028268
sample estimates:
mean of the differences 
                   11.4 



1. Uma distribuidora de combustíveis deseja verificar se um novo tipo de gasolina é eficaz na revitalização de motores velhos. Com esse objetivo,  seleciona 12 automóveis de um mesmo modelo com mais de 8 anos de uso e, após  regulagem de seus motores, verifica o consumo de combustível (em km/litro). Em  seguida, o carro é abastecido com o novo tipo de combustível durante 15  semanas, e uma nova aferição do consumo é feita. Defina as variáveis aleatórias Xi e Yi como o rendimento do automóvel i respectivamente antes e após as 15  semanas. Vemos que Xi e Yi foram medidas em uma mesma unidade amostral e,  assim, é razoável assumir que exista alguma dependência entre elas. Ao medir a característica de interesse em duas ocasiões, para cada uma das unidades  amostrais, pretende-se diminuir a influência de outros fatores e ressaltar um  possível efeito do tipo de gasolina no desempenho do veículo. Os dados coletados para os 12 automóveis são apresentados na tabela a seguir:

Utilizando um nível de significância de 5%, verifique se houve diferença no rendimento dos
veículos.


> Σd= 35,3
Σd2= 130,55
d = 2,941667
sd= 1,558238
Tp= 6,539586
valor p = 2x(0,0005)=0,001


In [113]:
%%R

antes  <- c(8.1, 7.9, 6.8, 7.8, 7.6, 7.9, 5.7, 8.4, 8, 9.5, 8, 6.8)
depois <- c(11.6, 8.8, 9.9, 9.5, 11.6, 9.1, 10.6, 10.8, 13.4, 10.6, 10.5, 11.4)
t.test(antes, depois, paired=TRUE, conf.level=0.95, alternative='less')


	Paired t-test

data:  antes and depois
t = -6.5396, df = 11, p-value = 0.00002097
alternative hypothesis: true difference in means is less than 0
95 percent confidence interval:
      -Inf -2.133833
sample estimates:
mean of the differences 
              -2.941667 



2. Dez cobaias foram submetidas ao tratamento de engorda com certa ração. Os pesos em gramas, antes e após o teste são dados a seguir (supõe-se que  provenham de distribuições normais). Considerando um nível de 1% de  significância, podemos concluir que existe diferença no peso médio dos animais antes e após o tratamento de engorda?

> Σd= -66
Σd2= 882
d = -6,6
sd= 7,04
Tp= -2,96
valor p = 2x(0,01)=0,02



In [114]:
%%R

antes  <- c(635, 704, 662, 560, 603, 745, 698, 575, 633, 669)
depois <- c(640, 712, 681, 558, 610, 740, 707, 585, 635, 682)
t.test(antes, depois, paired=TRUE, conf.level=0.99, alternative='two.sided')


	Paired t-test

data:  antes and depois
t = -2.9635, df = 9, p-value = 0.01587
alternative hypothesis: true difference in means is not equal to 0
99 percent confidence interval:
 -13.8377274   0.6377274
sample estimates:
mean of the differences 
                   -6.6 



## Teste t para duas amostras independentes

Caso 1: As duas populações parecem ter variâncias iguais (Homocedásticas)

Caso 2: As duas populações parecem ter variâncias desiguais (Heterocedásticas)

Exemplo 1:

Digitadores são treinados em uma empresa em duas turmas distintas. Na primeira, denominada turma J, utiliza-
se um método japonês de ensino, ao passo que na segunda turma, denominada turma A, utiliza-se um método

alemão. Deseja-se comparar os dois métodos e para tanto, 16 alunos de cada turma foram escolhidos
aleatoriamente e uma mesma tarefa foi atribuída a cada um. Ao final do experimento, o tempo gasto na realização
da tarefa, para cada aluno, foi anotado. No processo, dois computadores utilizados pelos alunos selecionados da
turma J e três da turma A apresentaram problemas que impediram a realização da tarefa; o tamanho da amostra foi
assim reduzido para 14 e 13, respectivamente, para as turmas J e A. Os dados obtidos encontram-se resumidos a
seguir. Considerando-se um nível de 5% de significância, verifique se há evidências suficientes para afirmar que os dois
métodos são equivalentes, ou seja, o tempo médio para as duas turmas são iguais.

| Turma | n | Média | Desvio padrão | 
| ----- | - | ----- | ------------- |
| J | 14 | 11,57 | 4,1 |
| A | 13 | 15,38 | 4,3 |

In [115]:
%%R

n_1 <- 14
n_2 <- 13
mean_1 <- 11.57
mean_2 <- 15.38
var_1 <- 4.1^2
var_2 <- 4.3^2
alpha <- 0.05

# 1. Avaliar se as populações são homocedásticas ou heterocedásticas (por meio do teste F). Onde:
# H0: var_1 = var_2 
# H1: var_1 <> var_2

f_test_to_var(n_1, n_2, var_1, var_2, alpha=alpha)

# Podemos concluir a um nível de 5% de significância que NÃO existe diferença na variância dos tempos das duas turmas estudadas (são homocedásticas).

# 2. Sabendo se a amostra é homocedástica ou heterocedástica é possível realizar o teste t para média de duas amostras. (Aplicar teste T correspondente)
# H0: mean_1 = mean_2 
# H1: mean_1 <> mean_2

two_sample_t_test_homocedastico(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=alpha)

F =  0.9091401 
Fl =  0.3171406 
Fr =  3.239263 
Resultado =  Aceita H0
df =  25 s_comb =  17.6164
t_stat =  -2.356786   t_critical =  2.059539 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.02657463 alpha = 0.05   Accept H1? R.:  TRUE
E =  3.329467 IC = [ -7.139467 ;  -0.4805331 ] Accept H1? R.:  TRUE

Exemplo 2:
Uma empresa avaliadora de imóveis está estudando as regiões central e oeste da cidade de São Paulo. O
objetivo principal é verificar se o preço médio (em R$10.000,00), praticado para imóveis comerciais de um dado
tamanho, é o mesmo nas duas áreas. Duas amostras aleatórias foram selecionadas de cada região. As medidas
resumo para esse estudo encontram-se a seguir. Considerando-se um nível de 5% de significância, o que você pode concluir?

| Região | n | Média | Desvio padrão |
| ---| --- | --- | --- |
| Central | 20 | 40,2 | 0,7 |
| Oeste | 15 | 36,7 | 1,9|

In [116]:
%%R

n_1 <- 20
n_2 <- 15
mean_1 <- 40.2
mean_2 <- 36.7
var_1 <- 0.7^2
var_2 <- 1.9^2
alpha <- 0.05

# 1. Avaliar se as populações são homocedásticas ou heterocedásticas (por meio do teste F). Onde:
# H0: var_1 = var_2 
# H1: var_1 <> var_2

f_test_to_var(n_1, n_2, var_1, var_2, alpha=alpha)

# Podemos concluir a um nível de 5% de significância que EXISTE diferença na variância dos tempos das duas turmas estudadas (são heterocedástica).

# 2. Sabendo se a amostra é homocedástica ou heterocedástica é possível realizar o teste t para média de duas amostras. (Aplicar teste T correspondente)
# H0: mean_1 = mean_2 
# H1: mean_1 <> mean_2

two_sample_t_test_heterocedastico(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=alpha)



F =  0.1357341 
Fl =  0.3777965 
Fr =  2.860722 
Resultado =  Aceita H1
t_stat =  6.796864   t_critical =  2.144787 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.000008640741 alpha = 0.05   Accept H1? R.:  TRUE
E =  1.104444 IC = [ 2.395556 ;  4.604444 ] Accept H1? R.:  TRUE

1. Uma nova técnica de impressão 3D foi desenvolvida. Para comparar a eficiência entre a técnica
tradicional e a nova técnica foi feita uma análise do tempo de secagem (em minutos). Uma amostra de
14 cubos impressos pelo método tradicional (método 1) apontou uma média de 121 minutos e um
desvio padrão de 16 minutos. 10 cubos impressos pelo novo método (método 2) indicaram uma média
de 112 minutos e um desvio padrão de 15 minutos. Ao nível de significância de 5%, podemos afirmar
que existe diferença nos tempos de secagem das duas técnicas? Assuma que os desvios padrões
populacionais são iguais.


> Hipótese bilateral \
Variância combinada = 243,3194 \
Estatística de teste = 1,39 \
Grau de liberdade = 22 \
Valor p = 0,2 -> Como o valor p é superior ao nível de significância (α) NÃO devemos rejeitar a hipótese nula, ou seja, pode-se afirmar com 5% de significância que não há diferença nos tempos médios de secagem das duas técnicas.

In [117]:
%%R

n_1 <- 14
n_2 <- 10
mean_1 <- 121
mean_2 <- 112
var_1 <- 16^2
var_2 <- 15^2
alpha <- 0.05

# 1. Avaliar se as populações são homocedásticas ou heterocedásticas (por meio do teste F). Onde:
# H0: var_1 = var_2 
# H1: var_1 <> var_2

f_test_to_var(n_1, n_2, var_1, var_2, alpha=alpha)

# Podemos concluir a um nível de 5% de significância que EXISTE diferença na variância dos tempos das duas turmas estudadas (são heterocedástica).

# 2. Sabendo se a amostra é homocedástica ou heterocedástica é possível realizar o teste t para média de duas amostras. (Aplicar teste T correspondente)
# H0: mean_1 = mean_2 
# H1: mean_1 <> mean_2

# two_sample_t_test_heterocedastico(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=alpha)
two_sample_t_test_homocedastico(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=alpha)

F =  1.137778 
Fl =  3.830596 
Fr =  0.3019294 
Resultado =  Aceita H1
df =  22 s_comb =  243.3182
t_stat =  1.393521   t_critical =  2.073873 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.1773818 alpha = 0.05   Accept H1? R.:  FALSE
E =  13.39402 IC = [ -4.394023 ;  22.39402 ] Accept H1? R.:  FALSE

2. Uma empresa realizou um estudo para avaliar o tempo médio de adaptação de funcionários recém
formados e de funcionários que atuam no mercado há mais tempo. Foram selecionados aleatoriamente
26 recém formados, obtendo-se um tempo médio de adaptação de 2,8 anos e um desvio padrão de 0,6
anos.; 26 funcionários que atuam no mercado há mais tempo, obtendo-se um tempo médio de
adaptação de 2,3 anos e um desvio padrão de 0,7 anos. Teste a hipótese de que existe diferença entre
os tempos médios de adaptação, considerando um nível de 5% de significância. Assuma que os desvios
padrões populacionais são iguais.

> Hipótese bilateral \
Variância combinada = 0,425 \
Estatística de teste = 2,77 \
Grau de liberdade = 50 \
Valor p = 2x0,005 = 0,010 -> Como o valor p é inferior ao nível de significância (α) devemos rejeitar a hipótese nula, ou seja, pode-se afirmar com 5% de significância que existe diferença entre os tempos médios de  adaptação. \
Intervalo de confiança = [0,133 ; 0,867] -> Como os limites do intervalo apresentam sinal positivo (+) conclui-se que o tempo médio
de adaptação dos recém formados é maior do que tempo médio de adaptação dos funcionários que atuam
no mercado há mais tempo.

In [118]:
%%R

n_1 <- 26
n_2 <- 26
mean_1 <- 2.8
mean_2 <- 2.3
var_1 <- 0.6^2
var_2 <- 0.7^2
alpha <- 0.05

# 1. Avaliar se as populações são homocedásticas ou heterocedásticas (por meio do teste F). Onde:
# H0: var_1 = var_2 
# H1: var_1 <> var_2

f_test_to_var(n_1, n_2, var_1, var_2, alpha=alpha)

# Podemos concluir a um nível de 5% de significância que NÃO EXISTE diferença na variância dos tempos das duas turmas estudadas (são heterocedástica).

# 2. Sabendo se a amostra é homocedástica ou heterocedástica é possível realizar o teste t para média de duas amostras. (Aplicar teste T correspondente)
# H0: mean_1 = mean_2 
# H1: mean_1 <> mean_2

two_sample_t_test_homocedastico(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=alpha)

F =  0.7346939 
Fl =  0.4483698 
Fr =  2.230302 
Resultado =  Aceita H0
df =  50 s_comb =  0.425
t_stat =  2.765332   t_critical =  2.008559 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.007944886 alpha = 0.05   Accept H1? R.:  TRUE
E =  0.3631679 IC = [ 0.1368321 ;  0.8631679 ] Accept H1? R.:  TRUE

3. Deseja-se saber se 2 máquinas de empacotar café estão fornecendo o mesmo peso médio em kg.
Extraem-se duas amostras, uma de cada máquina (supondo que os pesos das amostras sigam uma
distribuição normal):

Máquina Nova: 36 amostras, média = 0,81 kg, variância = 0,00020 kg2. \
Máquina Velha: 39 amostras, média = 0,78 kg, variância = 0,00135 kg2.

Qual é a sua conclusão a 5% de significância? Assuma que os desvios padrões populacionais são
diferentes.

> Hipótese bilateral \
Estatística de teste = 4,73 \
Grau de liberdade = 35 \
Valor p = 2x0,0005=0,0010 -> Como o valor p é inferior ao nível de significância (α) devemos rejeitar a hipótese nula, ou seja, pode-se afirmar com 5% de significância que há diferença nos pesos médios dos pacotes das duas máquinas. \
Intervalo de confiança: [0,0171 ; 0,0429] Como os limites do intervalo apresentam sinal positivo (+) conclui-se que o peso médio
dos pacotes de café empacotados pela máquina nova é maior do que aqueles empacotados pela máquina
velha.

In [119]:
%%R

n_1 <- 36
n_2 <- 39
mean_1 <- 0.81
mean_2 <- 0.78
var_1 <- 0.00020
var_2 <- 0.00135
alpha <- 0.05

# 1. Avaliar se as populações são homocedásticas ou heterocedásticas (por meio do teste F). Onde:
# H0: var_1 = var_2 
# H1: var_1 <> var_2

f_test_to_var(n_1, n_2, var_1, var_2, alpha=alpha)

# Podemos concluir a um nível de 5% de significância que EXISTE diferença na variância dos tempos das duas turmas estudadas (são heterocedástica).

# 2. Sabendo se a amostra é homocedástica ou heterocedástica é possível realizar o teste t para média de duas amostras. (Aplicar teste T correspondente)
# H0: mean_1 = mean_2 
# H1: mean_1 <> mean_2

two_sample_t_test_heterocedastico(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=alpha)

F =  0.1481481 
Fl =  0.5147002 
Fr =  1.92543 
Resultado =  Aceita H1
t_stat =  4.733313   t_critical =  2.030108 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.00003580874 alpha = 0.05   Accept H1? R.:  TRUE
E =  0.01286694 IC = [ 0.01713306 ;  0.04286694 ] Accept H1? R.:  TRUE

4. Os dados a seguir referem-se ao conteúdo médio de material sólido em suspensão (mg L-1) nas águas dos rios Verde e Crespo. O material sólido em suspensão difere nos dois rios? Utilize um nível de 5% de
significância e assuma que os desvios padrões populacionais são diferentes. Comente o resultado.

| Rio Verde | Rio Crespo |
| --- | --- |
| 210 | 410 |
| 242 | 390 |
| 226 | 501 |
| 268 | 420 |
| 251 | 480 |
| 206 | 456 |
| 218 | 495 |
| 215 | 507 |
| 207 | 385 |

> Rio Verde: n=9, média da amostra = 227 , s=21,9 \
Rio Crespo: n=9, média da amostra = 449,3 , s=48,9 \
Hipótese bilateral \
Estatística de teste = -12,45 \
Grau de liberdade = 8 \
Valor p = 2x0,0005=0,0010 Como o valor p é inferior ao nível de significância (α) devemos rejeitar a hipótese nula, ou seja, pode-se afirmar com 5% de  significância que há diferença no conteúdo médio de material sólido em suspensão (mg L-1) nas águas dos rios Verde e Crespo. \
Intervalo de confiança: Como os limites do intervalo apresentam sinal negativo (-) conclui-se que o conteúdo médio de material
sólido em suspensão (mg L-1) nas águas do rio Verde é menor do que nas águas do rio Crespo.

In [120]:
%%R

rio_verde  <- c(210, 242, 226, 268, 251, 206, 218, 215, 207)
rio_crespo <- c(410, 390, 501, 420, 480, 456, 495, 507, 385)

n_1 <- length(rio_verde)
n_2 <- length(rio_crespo)
mean_1 <- mean(rio_verde)
mean_2 <- mean(rio_crespo)
var_1 <- sd(rio_verde)^2
var_2 <- sd(rio_crespo)^2
alpha <- 0.05

cat('Rio Verde n=', n_1, 'média=', mean_1, 'sd=',sd(rio_verde))
cat('\nRio Crespo n=', n_2, 'média=', mean_2, 'sd=',sd(rio_crespo))
cat('\n')
# 1. Avaliar se as populações são homocedásticas ou heterocedásticas (por meio do teste F). Onde:
# H0: var_1 = var_2 
# H1: var_1 <> var_2

f_test_to_var(n_1, n_2, var_1, var_2, alpha=alpha)

# Podemos concluir a um nível de 5% de significância que EXISTE diferença na variância dos tempos das duas turmas estudadas (são heterocedástica).

# 2. Sabendo se a amostra é homocedástica ou heterocedástica é possível realizar o teste t para média de duas amostras. (Aplicar teste T correspondente)
# H0: mean_1 = mean_2 
# H1: mean_1 <> mean_2

two_sample_t_test_heterocedastico(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=alpha)

Rio Verde n= 9 média= 227 sd= 21.9032
Rio Crespo n= 9 média= 449.3333 sd= 48.90297
F =  0.2006063 
Fl =  0.2255676 
Fr =  4.43326 
Resultado =  Aceita H1
t_stat =  -12.44773   t_critical =  2.306004 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.000001620893 alpha = 0.05   Accept H1? R.:  TRUE
E =  41.18834 IC = [ -263.5217 ;  -181.145 ] Accept H1? R.:  TRUE

## Teste F para duas amostras (variâncias)

Um fabricante de esferas para rolamentos desenvolveu um novo método de produção, mais
barato. Entretanto, ele desconfia que os novos lotes apresentam variabilidade diferente daqueles
produzidos pelo método antigo (com relação ao diâmetro das esferas). Para cada método, ele
selecionou uma amostra aleatória. As medidas descritivas para cada método encontram-se a
seguir:

Método antigo: n1 = 15 ; x̅̅1̅ = 29,93 mm ; sd1 = 0,03 mm2

Método novo: n2 = 10 ; x̅̅2̅ = 29,89 mm ; sd2 = 0,19 mm2

Nesse exemplo vamos construir o teste bilateral para comparação de duas variâncias para um nível de significância de 5%.
Hipóteses estatísticas: 
- Ho: sd1 = sd2
- HA: sd1 ≠ sd2

In [121]:
#Rejeitar H0 se: F < Fl ou se F > Fr
%%R
f_test_to_var(n_1=15, n_2=10, var_1=0.03, var_2=0.19, alpha=0.05)

F =  0.1578947 
Fl =  0.3115944 
Fr =  3.797952 
Resultado =  Aceita H1

1. Um analista da qualidade quer avaliar se existe diferença na variabilidade da produção de
eixo comando desenvolvido por dois sistemas de usinagem. A tabela a seguir apresenta as
medidas descritivas de duas populações independentes com distribuição Normal.
Podemos dizer que as variâncias de ambas são iguais? Utilize um nível de significância de
5%.

> 1. F=0,2225
FL = 0,4527
FR = 2,154
Rejeitar H0 se: F < 0,4527 ou se F > 2,154
Como F < FL, rejeitamos a hipótese nula.

In [122]:
#Rejeitar H0 se: F < Fl ou se F > Fr
%%R

var_1 <- 1.3623^2
var_2 <- 2.8876^2
f_test_to_var(n_1=25, n_2=30, var_1, var_2, alpha=0.05)

F =  0.2225725 
Fl =  0.4509699 
Fr =  2.154006 
Resultado =  Aceita H1

2. Um consultor de saúde deseja comparar os índices de satisfação dos pacientes de dois
hospitais. O consultor coleta as classificações de 20 pacientes para cada um dos hospitais.
No hospital A a variância observada foi igual a 66,958 e no hospital B foi de 154,537.
Execute um teste de duas variâncias para determinar se os desvios-padrão nas avaliações
dos pacientes dos dois hospitais são diferentes. Utilize um nível de significância de 5%.

> F=0,4333
FL = 0,3959
FR = 2,526
Rejeitar H0 se: F < 0,3959 ou se F > 2,526
Não rejeitamos a hipótese nula.

In [123]:
#Rejeitar H0 se: F < Fl ou se F > Fr
%%R
f_test_to_var(n_1=20, n_2=20, var_1=66.958, var_2=154.537, alpha=0.05)

F =  0.4332814 
Fl =  0.3958122 
Fr =  2.526451 
Resultado =  Aceita H0

## Teste unidade 4

1. Uma revista de viagens de negócios quer classificar os aeroportos internacionais de acordo com a avaliação média da população de pessoas que viajam a negócios. Será usada uma escala de classificação, sendo 0 uma avaliação baixa e 10 uma avaliação elevada, e os aeroportos que receberem uma avaliação média populacional maior que 7 serão designados como aeroportos com um atendimento de alto nível. A equipe da revista pesquisou uma amostra de 27 viajantes de negócios em cada aeroporto para obter os dados da avaliação. A amostra do aeroporto de Londres produziu uma avaliação média igual a 7,25 com desvio padrão igual a 1,052. Os dados indicam que o aeroporto de Londres deveria ser designado como um aeroporto de alto nível? Utilize um nível de 5% de significância.

In [124]:
%%R
mean_0 <- 7
n <- 27
mean_sample <- 7.25
sd <- 1.052
alpha <- 0.05

# H0: mean_sampĺe <= 7
# H1: mean_sampĺe > 7

one_sample_t_test(n, sd, mean_sample, mean_0, h1_signal='>', alpha=alpha)


df =  26
t_stat =  1.234827   t_critical =  1.705618 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.1139692 alpha = 0.05   Accept H1? R.:  FALSE
E =  0.3453151 IC = [ 6.654685 ;  7.345315 ] Accept H1? R.:  FALSE

In [125]:
%%R
mean_0 <- 7
n <- 27
mean_sample <- 7.25
sd <- 1.052
alpha <- 0.05

# H0: mean_sampĺe <= 7
# H1: mean_sampĺe > 7

one_sample_z_test(n, sd, mean_sample, mean_0, h1_signal='>', alpha=alpha)


z_stat =  1.234827   z_critical =  1.644854 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.1084474 alpha = 0.05   Accept H1? R.:  FALSE
E =  0.3330129 IC = [ 6.916987 ;  7.583013 ] Accept H1? R.:  FALSE

2. As companhias de seguro de automóvel estão cogitando elevar os prêmios para aqueles que falam ao telefone enquanto dirigem. Um grupo de defesa dos consumidores alega que este problema não é tão sério, porque menos de 10% dos motoristas usam o telefone. Uma companhia de seguro faz uma pesquisa e constata que, entre 500 motoristas selecionados aleatoriamente, 72 usam o telefone. Teste a afirmação do grupo de consumidores ao nível de 2% de significância.

In [126]:
%%R
pi_0 <- 0.10
n <- 500
pi <- 72/500
alpha <- 0.02

# H0: mean_sampĺe >= 10
# H1: mean_sampĺe < 10

one_sample_z_proportion_test(n, pi, pi_0, h1_signal='<', alpha=alpha)

n * pi > 5 =  TRUE
n * (1 - pi) > 5 =  TRUE

z_stat =  3.279566   z_critical =  -2.053749 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.9994802 alpha = 0.02   Accept H1? R.:  FALSE
E =  0.03224634 IC = [ 0.1117537 ;  0.1762463 ] Accept H1? R.:  TRUE

3. Um banco realiza um estudo idealizado para identificar as diferenças na utilização das contas correntes pelos clientes em duas de suas filiais. Uma amostra aleatória de 28 contas correntes é selecionada da filial situada em CG e uma amostra aleatória independente de 22 contas correntes é selecionada da sua filial em BM. O saldo atual da conta corrente é registrado para cada uma das contas. A seguir temos um resumo dos saldos bancários:

| | Filial CG | Filial BM | 
|-- | -- | -- |
| Média amostral | 1025 | 910 |
| Desvio padrão amostral | 150 | 125 |

O saldo médio das contas correntes mantidas pela população de clientes difere entre as duas filiais? Utilize um nível de 5% de significância.

In [127]:
%%R
n_1 <- 28
n_2 <- 22
mean_1 <- 1025
mean_2 <- 910
sd_1 <- 150
sd_2 <- 125
var_1 <- sd_1^2
var_2 <- sd_2^2
alpha <- 0.5

# H0: mean_1 = mean_2
# H1: mean_1 <> mean_2


# 1. Avaliar se as populações são homocedásticas ou heterocedásticas (por meio do teste F). Onde:
# H0: var_1 = var_2 
# H1: var_1 <> var_2

f_test_to_var(n_1, n_2, var_1, var_2, alpha=alpha)

# Podemos concluir a um nível de 5% de significância que EXISTE diferença na variância dos tempos das duas turmas estudadas (são homocedásticas).

# 2. Sabendo se a amostra é homocedástica ou heterocedástica é possível realizar o teste t para média de duas amostras. (Aplicar teste T correspondente)
# H0: mean_1 = mean_2 
# H1: mean_1 <> mean_2

two_sample_t_test_heterocedastico(n_1, n_2, mean_1, mean_2, var_1, var_2, h1_signal='<>', alpha=alpha)


F =  1.44 
Fl =  1.336221 
Fr =  0.7617316 
Resultado =  Aceita H1
t_stat =  2.955723   t_critical =  0.686352 Accept H1? R.:  TRUE
p_value <= alpha? p-value = 0.007546773 alpha = 0.5   Accept H1? R.:  TRUE
E =  26.70429 IC = [ 88.29571 ;  141.7043 ] Accept H1? R.:  TRUE

Um experimento (hipotético) sobre o efeito do álcool na habilidade perceptual motora é conduzido. 10 indivíduos são testado duas vezes, uma depois de ter tomado dois drinks e uma depois de tomado dois copos de água. Os dois testes foram realizados em dois dias diferentes para evitar influência do efeito do álcool. Metade dos indivíduos tomou a bebida alcoólica primeiro e a outra metade água. Os escores dos 10 indivíduos são mostrados abaixo. Escores mais altos refletem uma melhor performance. 

Indivíduo	1	2	3	4	5	6	7	8	9	10

Água	16	15	11	20	19	14	13	15	14	16

Álcool	13	13	12	16	16	11	10	15	9	16

Deseja-se testar se houve alteração na habilidade perceptual motora mediante as duas bebidas testadas. Utilize um nível de significância de 1%.

In [128]:
%%R

agua  <- c(16, 15, 11, 20, 19, 14, 13, 15, 14, 16)
alcool <- c(13, 13, 12, 16, 16, 11, 10, 15, 9, 16)
t.test(agua, alcool, paired=TRUE, conf.level=0.99, alternative='two.sided')


	Paired t-test

data:  agua and alcool
t = 3.6006, df = 9, p-value = 0.005743
alternative hypothesis: true difference in means is not equal to 0
99 percent confidence interval:
 0.2143177 4.1856823
sample estimates:
mean of the differences 
                    2.2 



Em um concurso público promovido por uma empresa estatal, os candidatos às vagas de Engenharia constituem a nossa população de interesse. Entre eles, os que se submeteram a uma preparação específica para o concurso constituem a subpopulação A e os que não fizeram essa preparação constituem a sub-população B. Foram coletadas amostras aleatórias em ambas as sub-populações e os resultados obtidos foram os seguintes:

| Sub-população | Tamanho amostral | Aprovados |
| -- | -- | -- |
| Prepararam-se (A) | 140 | 34 |
| Não se prepararam (B) | 230 | 53 |

Pode-se dizer que houve diferença na proporção de aprovados entre as duas sub-populações estudadas? Utilize um nível de 10% de significância.

In [129]:
%%R


two_sample_z_proportion_test(n_1=140, n_2=230, p_1=34/140, p_2=53/230, h1_signal='<>', alpha=0.1)

p_^ =  0.2351351 

z_stat =  0.2732625   z_critical =  1.644854 Accept H1? R.:  FALSE
p_value <= alpha? p-value = 0.7846514 alpha = 0.1   Accept H1? R.:  FALSE
E =  0.07477412 IC = [ -0.06235176 ;  0.08719648 ] Aceept H1? R.:  FALSE